----------------------------
# Session 2 
----------------------------

In [1]:
%matplotlib inline
import torch
import numpy as np
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f"Device : {device}")

Device : cpu


## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [2]:
import torch.nn as nn

### Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [3]:
from torch.utils.data import TensorDataset

In [4]:
#@title  the dataset as simple tensors
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [5]:
# Define the dataset based on the Dataset interface 
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [6]:
from torch.utils.data import DataLoader

In [7]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

We can use the data loader in a `for` loop. Let's look at an example.

In [8]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 68.,  96.,  71.],
        [101.,  44.,  37.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [103.,  43.,  36.]])
tensor([[104., 118.],
        [ 21.,  38.],
        [ 22.,  37.],
        [103., 119.],
        [ 20.,  38.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

### nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [9]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.0384, -0.2567, -0.1555],
        [-0.0552,  0.2166,  0.1584]], requires_grad=True)
Parameter containing:
tensor([0.4452, 0.5586], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [10]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0384, -0.2567, -0.1555],
         [-0.0552,  0.2166,  0.1584]], requires_grad=True),
 Parameter containing:
 tensor([0.4452, 0.5586], requires_grad=True)]

We can use the model to generate predictions in the same way as before.

In [11]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-20.6352,  17.8490],
        [-28.5992,  24.7297],
        [-39.6264,  33.9636],
        [-12.4297,  10.0981],
        [-32.4296,  28.6283],
        [-20.3402,  17.5772],
        [-28.4980,  24.6715],
        [-39.7435,  34.0668],
        [-12.7247,  10.3699],
        [-32.6234,  28.8420],
        [-20.5340,  17.7908],
        [-28.3042,  24.4578],
        [-39.7276,  34.0218],
        [-12.2358,   9.8844],
        [-32.7246,  28.9002]], grad_fn=<AddmmBackward0>)

### Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [12]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities. 

In [13]:
# Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [14]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(8961.2520, grad_fn=<MseLossBackward0>)


### Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group. Since the group can be larger than a single element, it corresponds to SGD with _minibatch_. In fact, the `optimizer` is not responsible for loading batches, as this aspect is handled by the `DataLoader`.

In [15]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Exercise 1: Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [16]:
# Utility function to train the model
def train_with_MiniBatchSGD(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            raise Exception('Compute the gradient')
            
            # 4. Update parameters using gradients
            raise Exception('Use the optimizer to update the parameters')
            
            # 5. Reset the gradients to zero
            raise Exception('reset the gradients using the optimzer')
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [17]:
train_with_MiniBatchSGD(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 791.2788
Epoch [20/100], Loss: 307.6425
Epoch [30/100], Loss: 85.2111
Epoch [40/100], Loss: 213.4459
Epoch [50/100], Loss: 155.3407
Epoch [60/100], Loss: 59.4472
Epoch [70/100], Loss: 42.0978
Epoch [80/100], Loss: 88.9277
Epoch [90/100], Loss: 49.2883
Epoch [100/100], Loss: 61.3160


Let's generate predictions using our model and verify that they're close to our targets.

In [18]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.0763,  72.2417],
        [ 80.9597,  96.4847],
        [119.2587, 139.9561],
        [ 27.0622,  47.3321],
        [ 96.2716, 105.7728],
        [ 56.9491,  71.2566],
        [ 80.5358,  95.7134],
        [119.4434, 140.1388],
        [ 28.1894,  48.3172],
        [ 96.9748, 105.9866],
        [ 57.6523,  71.4704],
        [ 79.8325,  95.4996],
        [119.6826, 140.7274],
        [ 26.3589,  47.1183],
        [ 97.3988, 106.7579]], grad_fn=<AddmmBackward>)

In [19]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [20]:
with torch.no_grad():
    print(model(torch.tensor([[75, 63, 44.]])))

tensor([[54.5299, 68.5306]])


## Training an MLP classifier on FashionMNIST
FashionMNIST is a classification benchmark, where images are associated to classes. In this exercise, you shall have to adapt the linear regression to this context. The main adaptation steps are as follows:
* Change the dataset (using TorchVision)
* Replace the mere linear model by an MLP
* Change the MSE loss to a CrossEntropy loss


### Working with data
Remember that PyTorch has two `primitives to work with data <https://pytorch.org/docs/stable/data.html>`_:
``torch.utils.data.DataLoader`` and ``torch.utils.data.Dataset``.
``Dataset`` stores the samples and their corresponding labels, and ``DataLoader`` wraps an iterable around
the ``Dataset``.

PyTorch offers domain-specific libraries such as `TorchText <https://pytorch.org/text/stable/index.html>`_,
`TorchVision <https://pytorch.org/vision/stable/index.html>`_, and `TorchAudio <https://pytorch.org/audio/stable/index.html>`_,
all of which include datasets. For this tutorial, we  will be using a TorchVision dataset.

The ``torchvision.datasets`` module contains ``Dataset`` objects for many real-world vision data like
CIFAR, COCO (`full list here <https://pytorch.org/vision/stable/datasets.html>`_). In this tutorial, we
use the FashionMNIST dataset. 
Each training and test example is assigned to one of the following labels:

| Label | Description |
| --- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |



Every TorchVision ``Dataset`` includes an argument: ``transform``
 to modify the samples.




In [17]:
# Import TorchVision utilities
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

We pass the ``Dataset`` as an argument to ``DataLoader``. This wraps an iterable over our dataset, and supports
automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element
in the dataloader iterable will return a batch of 64 features and labels.



In [18]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# Get one batch of data and print its tensor shape
X,y = next(iter(train_dataloader))
print("Shape of X [N, C, H, W]: ", X.shape)
print("Shape of y: ", y.shape, y.dtype)

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


--------------




## Exercise 2: Creating  the MLP model

To define a MLP, you may rely on `nn.Sequential <https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html>`. It implements a sequential container modules made from a sequence of `nn.Module`. The goal here is to have the following sequence:

$$
I\in\mathbb R^{28}\otimes\mathbb R^{28}  
\xrightarrow{\mathrm{flatten}}{} \mathbb R^{28\times 28}=\mathbb R^{784}
\xrightarrow{\mathrm{Linear+ReLU}}{} \mathbb R^{512}
\xrightarrow{\mathrm{Linear+ReLU}}{} \mathbb R^{512}
\xrightarrow{\mathrm{Linear}}{} \mathbb R^{n_c}
$$

You shall implement this archticture, and determine what is the logical value for $n_c$ in our settings.

In [19]:
debug = False
def create_MLP():
    model = nn.Sequential().to(device)
    return model

def test_exercise2():
    model = create_MLP()
    if not isinstance(model, nn.Sequential):
        print('Exercise 2: [NOK]')
        if debug:
            print(f'The model should be a nn.Sequential instance not {type(model)}')
        return

    if len(model) != 6:
        print('Exercise 2: [NOK]')
        if debug:
            print(f'The model should be 6 layers not {len(mode)}')
        return
    
    model_device = next(model.parameters()).device
    expected_device = torch.empty(1).to(device).device
    if model_device != expected_device:
        print('Exercise 2: [NOK]')
        if debug:
            print(f'The model should run on {expected_device} not on {model_device}')
        return
    
    X,y = next(iter(train_dataloader))
    pred = model(X.to(device))
    expected_shape = torch.Size((batch_size, 10))
    if pred.shape != expected_shape:
        print('Exercise 2: [NOK]')
        if debug:
            print(f'The model predictions should be shaped as {expected_shape} not as {pred.shape}')
        return

    print('Exercise 2: [OK]')
    
    if debug:
        print(model)
  
test_exercise2()

Exercise 2: [NOK]


--------------




## Exercise 3: Defining the loss function.
To train a model, we need a `loss function <https://pytorch.org/docs/stable/nn.html#loss-functions>`. 
The MSE loss is not the usually selected one for classification problems. You should propose a better suited loss.



In [23]:
def create_loss():
    loss_fn = None
    return loss_fn

def test_exercise3():
    loss_fn = create_loss()
    if loss_fn is None:
        print('Exercise 3: [NOK]')
        if debug:
            print(f'The loss should be {expected_loss} not as {loss}')
        return
    y = torch.arange(1,dtype=torch.long).to(device)
    pred = torch.ones((1,10),dtype=float).to(device)
    loss = loss_fn(pred, y).item()
    expected_loss = np.log(10) # entropy of a random [0-9] source
    if abs(loss-expected_loss) > 1e-5:
        print('Exercise 3: [NOK]')
        if debug:
            print(f'The loss should be {expected_loss} not as {loss}')
        return

    print('Exercise 3: [OK]') 

test_exercise3()


Exercise 3: [NOK]


### Exercise 4: Performing the optimization
Getting inspired by the [Linear Regression example](#scrollTo=s4G-FSTAnTT8), complete the following function (single epoch) and launch the whole training procedure (remaining cells).


In [34]:
def perform_one_epoch(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    raise Exception('You need to complete the epoch update routine')

We also check the model's performance against the test dataset to ensure it is learning.



In [26]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*). During each epoch, the model learns
parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the
accuracy increase and the loss decrease with every epoch.



In [27]:
epochs = 10
model = create_MLP()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
loss_fn = create_loss()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    perform_one_epoch(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303956  [    0/60000]
loss: 2.293683  [ 6400/60000]
loss: 2.284105  [12800/60000]
loss: 2.276086  [19200/60000]
loss: 2.258217  [25600/60000]
loss: 2.236558  [32000/60000]
loss: 2.239137  [38400/60000]
loss: 2.213353  [44800/60000]
loss: 2.210768  [51200/60000]
loss: 2.178315  [57600/60000]
Test Error: 
 Accuracy: 45.3%, Avg loss: 2.174318 

Epoch 2
-------------------------------
loss: 2.184904  [    0/60000]
loss: 2.175675  [ 6400/60000]
loss: 2.128336  [12800/60000]
loss: 2.141005  [19200/60000]
loss: 2.095470  [25600/60000]
loss: 2.042057  [32000/60000]
loss: 2.070466  [38400/60000]
loss: 2.000316  [44800/60000]
loss: 2.006389  [51200/60000]
loss: 1.936662  [57600/60000]
Test Error: 
 Accuracy: 53.8%, Avg loss: 1.932366 

Epoch 3
-------------------------------
loss: 1.967941  [    0/60000]
loss: 1.937356  [ 6400/60000]
loss: 1.829858  [12800/60000]
loss: 1.860887  [19200/60000]
loss: 1.761983  [25600/60000]
loss: 1.711195  [32000/600

### Exercise 5: Variants on Fashion Mnist
In this exercise you will replace the MLP model by variants of the following CNN architecture.
$$
I\in\mathbb R^{28}\otimes\mathbb R^{28}  
\xrightarrow{\mathrm{Conv3x3+ReLU+Pool}}{} \mathbb R^{14}\otimes\mathbb R^{14}\otimes\mathbb R^{64}
\xrightarrow{\mathrm{Conv3x3+ReLU+Pool}}{} \mathbb R^{7}\otimes\mathbb R^{7}\otimes\mathbb R^{64}
\xrightarrow{\mathrm{Conv3x3+ReLU+Flatten}}{} \mathbb R^{490}
\xrightarrow{\mathrm{Linear}}{} \mathbb R^{n_c}
$$

You may consider the following variants:
* Max vs Avg Pooling,
* changing the padding of the convolutions,
* changing 3x3 conv by larger or smaller convolutions
* Playing with the number of channels of the hidden layers

You will compare the different variants in terms of :
* Classification accuracy
* Computation throughput
* Memory footprint both in terms of number of parameters, and activation maps